# AutoGen Basisvoorbeeld

In dit codevoorbeeld gebruik je het [AutoGen](https://aka.ms/ai-agents/autogen) AI Framework om een eenvoudige agent te maken.

Het doel van dit voorbeeld is om je de stappen te laten zien die we later zullen gebruiken in de aanvullende codevoorbeelden bij het implementeren van de verschillende agentpatronen.


## Importeer de benodigde Python-pakketten


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## Maak de Client

In dit voorbeeld gebruiken we [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) voor toegang tot de LLM.

Het `model` is gedefinieerd als `gpt-4o-mini`. Probeer het model te wijzigen naar een ander model dat beschikbaar is in de GitHub Models-marktplaats om de verschillende resultaten te zien.

Als snelle test voeren we gewoon een eenvoudige prompt uit - `Wat is de hoofdstad van Frankrijk`.


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## De Agent Definiëren

Nu we de `client` hebben ingesteld en bevestigd dat deze werkt, gaan we een `AssistantAgent` maken. Elke agent kan worden toegewezen:  
**naam** - Een korte naam die handig is om naar te verwijzen in multi-agent processen.  
**model_client** - De client die je in de vorige stap hebt gemaakt.  
**tools** - Beschikbare tools die de Agent kan gebruiken om een taak uit te voeren.  
**system_message** - De metaprompt die de taak, het gedrag en de toon van de LLM definieert.  

Je kunt de system message aanpassen om te zien hoe de LLM reageert. We behandelen `tools` in Les #4.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## Voer de Agent uit

De onderstaande functie zal de agent uitvoeren. We gebruiken de methode `on_message` om de status van de Agent bij te werken met het nieuwe bericht.

In dit geval werken we de status bij met een nieuw bericht van de gebruiker, namelijk `"Plan me a great sunny vacation"`.

Je kunt de inhoud van het bericht aanpassen om te zien hoe de LLM anders reageert.


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**Disclaimer**:  
Dit document is vertaald met behulp van de AI-vertalingsservice [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we streven naar nauwkeurigheid, dient u zich ervan bewust te zijn dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in de oorspronkelijke taal moet worden beschouwd als de gezaghebbende bron. Voor kritieke informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
